# Pydantic - data validation using python type annotations

Introductory tutorial to `pydantic` based on the documentation at [https://pydantic-docs.helpmanual.io/](https://pydantic-docs.helpmanual.io/).

## Example
Definition of pydantic classes by inheriting from `BaseModel`

In [1]:
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel
from rich.jupyter import print


class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: List[int] = []


external_data = {
    'id': '123',
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, 2, '3'],
}

What's going on here:

- `id` is of type int; the annotation-only declaration tells pydantic that this field is required. Strings, bytes or floats will be coerced to ints if possible; otherwise an exception will be raised.
- `name` is inferred as a string from the provided default; because it has a default, it is not required.

- `signup_ts` is a datetime field which is not required (and takes the value None if it's not supplied). pydantic will process either a unix timestamp int (e.g. 1496498400) or a string representing the date & time.

- `friends` uses python's typing system, and requires a list of integers. As with id, integer-like objects will be converted to integers.

### Parse and validate external data
If validation fails pydantic will raise an error with a breakdown of what was wrong:

In [2]:
user = User(**external_data)
print(user.id)

123

In [3]:
print(repr(user.signup_ts))

datetime.datetime(2019, 6, 1, 12, 22)

In [4]:
print(user.friends)

[1, 2, 3]

In [5]:
print(user.dict())

{
    'id': 123,
    'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),
    'friends': [1, 2, 3],
    'name': 'John Doe'
}

## Data validatation
Pydantic comes with internal validation providing high level warnings. Custom validation function can be implemented.

In [6]:
from pydantic import ValidationError

try:
    User(signup_ts='broken', friends=[1, 2, 'not number'])
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "id"
    ],
    "msg": "field required",
    "type": "value_error.missing"
  },
  {
    "loc": [
      "signup_ts"
    ],
    "msg": "invalid datetime format",
    "type": "value_error.datetime"
  },
  {
    "loc": [
      "friends",
      2
    ],
    "msg": "value is not a valid integer",
    "type": "type_error.integer"
  }
]

## JSON schema
Pydantic allows auto creation of JSON Schemas from models:

In [7]:
print(User.schema_json(indent=2))

{
  "title": "User",
  "type": "object",
  "properties": {
    "id": {
      "title": "Id",
      "type": "integer"
    },
    "signup_ts": {
      "title": "Signup Ts",
      "type": "string",
      "format": "date-time"
    },
    "friends": {
      "title": "Friends",
      "default": [],
      "type": "array",
      "items": {
        "type": "integer"
      }
    },
    "name": {
      "title": "Name",
      "default": "John Doe",
      "type": "string"
    }
  },
  "required": [
    "id"
  ]
}